# Implementing the `MiniBach` model

## Part 3: Training the network

In this step, we take the one-hot-encoded input and output vectors from Part 2 to train the neural network.

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import music21

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

We load the one-hot-encoded vectors (computed in the last part of this tutorial).

In [4]:
x = np.load('input.npy')
y = np.load('output.npy')

input_length = x.shape[1]
output_length = y.shape[1]

The `MiniBach` architecture consists of
- An input layer with 1664 units (originally 1344, but we extended it in Part 2, because the range of notes was not enough)
- A hidden layer of 200 units
- An output layer with 4928 units (originally 4480, same argument as the input layer)

The output layer will be separated into three chunks of data, one for each part (alto, tenor, and bass). Within those chunks, we have the predictions corresponding to each of the 64 sixteenth-note timesteps.

As this is a multi multi-class classification problem, the activation layer of the output is a `sigmoid` function and it is trained using a binary loss.

In [5]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(input_length,)),    
    keras.layers.Dense(200, activation='relu'),    
    keras.layers.Dense(output_length, activation='sigmoid')
])

The optimizer of the model is not discussed in the book.

Here, we use the `adam` optimizer.

In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [7]:
model.fit(x, y, epochs=10)

Epoch 1/10
105/105 [==============================] - 1s 8ms/step - loss: 0.1274 - binary_accuracy: 0.9727
Epoch 2/10
105/105 [==============================] - 1s 8ms/step - loss: 0.0558 - binary_accuracy: 0.9846
Epoch 3/10
105/105 [==============================] - 1s 8ms/step - loss: 0.0539 - binary_accuracy: 0.9846
Epoch 4/10
105/105 [==============================] - 1s 8ms/step - loss: 0.0523 - binary_accuracy: 0.9847
Epoch 5/10
105/105 [==============================] - 1s 8ms/step - loss: 0.0512 - binary_accuracy: 0.9848
Epoch 6/10
105/105 [==============================] - 1s 8ms/step - loss: 0.0504 - binary_accuracy: 0.9849
Epoch 7/10
105/105 [==============================] - 1s 8ms/step - loss: 0.0497 - binary_accuracy: 0.9850
Epoch 8/10
105/105 [==============================] - 1s 8ms/step - loss: 0.0489 - binary_accuracy: 0.9851
Epoch 9/10
105/105 [==============================] - 1s 8ms/step - loss: 0.0482 - binary_accuracy: 0.9852
Epoch 10/10
105/105 [================

In [8]:
model.save('trained_model.h5')

We have successfully trained and saved the model!

At this point, we are ready for generating some chorales! 

But before that, let's try predicting the accompaniment for one of the training examples.

In [9]:
soprano = x[200]
predictions = model.predict(soprano.reshape(1, -1))

predictions = predictions.reshape(-1)

soprano = soprano.reshape(64, -1)
alto = predictions[:1536].reshape(64, -1)
tenor = predictions[1536:3008].reshape(64, -1)
bass = predictions[3008:4928].reshape(64, -1)

music = {
    'soprano': soprano,
    'alto': alto,
    'tenor': tenor,
    'bass': bass
}

Let's try example 200. 

After obtaining the predictions from the model, we split the chunks of data into their corresponding part and `reshape` the arrays to access their 64-timesteps. 

In [10]:
SOPRANO_MIN = 57
SOPRANO_MAX = 81

ALTO_MIN = 52
ALTO_MAX = 74

TENOR_MIN = 48
TENOR_MAX = 69

BASS_MIN = 36
BASS_MAX = 64

ranges = {
    'soprano': {midinumber: (midinumber - SOPRANO_MIN + 1) for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {midinumber: (midinumber - ALTO_MIN + 1) for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {midinumber: (midinumber - TENOR_MIN + 1) for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {midinumber: (midinumber - BASS_MIN + 1) for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

reverse_ranges = {
    'soprano': {(midinumber - SOPRANO_MIN + 1): midinumber for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {(midinumber - ALTO_MIN + 1): midinumber for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {(midinumber - TENOR_MIN + 1): midinumber for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {(midinumber - BASS_MIN + 1): midinumber for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

In [11]:
def decode_note(n, rang):
    if n == 0:
        ret = '--'
    else:
        note = music21.note.Note(type='16th')        
        note.pitch.midi = reverse_ranges[rang][n]        
        ret = note
    return ret

In [12]:
generation = {
    'soprano': [],
    'alto': [],
    'tenor': [],
    'bass': []
}

for sixteenth in range(64):
    for part, notes in music.items():
        this_note = decode_note(np.argmax(notes[sixteenth]), part)
        if this_note == '--':
            last_note = generation[part][-1]
            this_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
            if last_note.tie:
                this_note.tie = music21.tie.Tie('continue')
            else:
                last_note.tie = music21.tie.Tie('start')
                generation[part][-1] = last_note
                this_note.tie = music21.tie.Tie('continue')
        else:
            if sixteenth > 0:
                last_note = generation[part][-1]
                if last_note.tie:
                    last_note.tie = music21.tie.Tie('stop')
        generation[part].append(this_note)    

In [13]:
df = pd.DataFrame(generation)

In [14]:
df

,soprano,alto,tenor,bass
0,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
1,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
2,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
3,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
4,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
5,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
6,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
7,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
8,<music21.note.Note B->,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->
9,<music21.note.Note B->,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note B->


In [15]:
s = music21.stream.Stream()
s.append(df.soprano.to_list())
a = music21.stream.Stream()
a.append(df.alto.to_list())
t = music21.stream.Stream()
t.append(df.tenor.to_list())
b = music21.stream.Stream()
b.append(df.bass.to_list())
stream = music21.stream.Stream([s,a,t,b])

In [16]:
stream.write('musicxml', 'example.musicxml')

'/home/napulen/dev/MiniBach/example.musicxml'

After this step, we have trained and saved a `MiniBach` model. 

We also used the model to predict the values of one of the training examples.

The next step is to try the model on an arbitrary melody. We will do that in the last part.